# Manually recorded scores for settling - see how much of each version

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import run_yasa
import logging
import mne
import yasa
import os
import argparse
import os

import mne
import numpy as np
import pandas as pd
from brainflow.board_shim import BoardShim, BoardIds
from brainflow.data_filter import DataFilter
log = lambda msg: logging.info(msg)

In [2]:
from models.eeg_states.eeg_states import load_and_prepare_all_events

# This already debounces them, e.g. keeps only _long and removes the trigger
all = load_and_prepare_all_events()

C:\Users\graha\.conda\envs\tf\lib\site-packages\google\cloud\firestore_v1\base_collection.py:300: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:130: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-10-09 20:17:27.992000+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  events.at[i, 'since'] = since_time
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:132: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  events['duration'] = events['timestamp'] - events['since']


In [3]:
all

,event,note,version,phase,source,BaseEvent,Preserve_SettlingEvent,Preserve_SettlingEventVersion,duration,TimestampUK,SinceUK,dayAndNightOf
0,tired_long,NaN,NaN,NaN,NaN,tired,tired,NaN,0 days 00:05:00,2024-10-09 21:22:27.992000+01:00,2024-10-09 21:17:27.992000+01:00,2024-10-09
1,tired,NaN,NaN,NaN,NaN,tired,tired,NaN,NaT,2024-10-09 21:28:57.046000+01:00,NaT,2024-10-09
2,tired,NaN,NaN,NaN,NaN,tired,tired,NaN,NaT,2024-10-09 21:32:36.824000+01:00,NaT,2024-10-09
3,tired_long,NaN,NaN,NaN,NaN,tired,tired,NaN,0 days 00:00:15.951000,2024-10-09 21:32:52.775000+01:00,2024-10-09 21:32:36.824000+01:00,2024-10-09
4,tired,NaN,NaN,NaN,NaN,tired,tired,NaN,NaT,2024-10-09 21:33:13.921000+01:00,NaT,2024-10-09
...,...,...,...,...,...,...,...,...,...,...,...,...
3742,great,,v2,mood,NaN,great,great,v2,NaT,2024-12-13 12:11:07.831000+00:00,NaT,2024-12-13
3743,great,,v4,day,NaN,great,great,v4,NaT,2024-12-13 12:26:10.823000+00:00,NaT,2024-12-13
3744,great,,v2,mood,NaN,great,great,v2,NaT,2024-12-13 12:26:12.331000+00:00,NaT,2024-12-13
3745,tired but,,v4,day,NaN,tired but,tired but,v4,NaT,2024-12-13 12:36:18.088000+00:00,NaT,2024-12-13


In [4]:
all['event'].unique()

array(['tired_long', 'tired', 'wired', 'wired_long', 'mid', 'mid_long',
       'sleepy', 'sleepy_long', 'struggling', 'struggling_long',
       'near worst', 'okish', 'okish_long', 'standard tired',
       'standard tired_long', 'lockable', 'great', 'great_long',
       'certain', 'irritated ', 'irritated _long', 'no', 'no_long',
       'maybe_long', 'maybe', 'lockable_long', 'somewhat foggy_long',
       'somewhat foggy', 'near worst_long', 'almost certain_long',
       'tired but wired_long', 'tired but wired', 'shakey', 'shakey_long',
       'irritated_long', 'angry', 'irritated', 'angry_long',
       'sub-optimal', 'sub-optimal_long', 'other_long', 'other',
       'reasonable enough_long', 'reasonable enough', 'neutral_long',
       'neutral', 'reasonable', 'reasonable_long', 'meh', 'meh_long',
       'tired but', 'overwhelmed', 'tired but_long', 'awake',
       'awake_long', 'alert', 'alert_long', 'relaxed', 'relaxed_long',
       'stubborn_brain_long', 'blink', 'certain_long', 't

In [5]:
all['BaseEvent'].unique()

array(['tired', 'wired', 'mid', 'sleepy', 'struggling', 'near worst',
       'okish', 'standard tired', 'lockable', 'great', 'certain',
       'irritated ', 'no', 'maybe', 'somewhat foggy', 'almost certain',
       'tired but wired', 'shakey', 'irritated', 'angry', 'sub-optimal',
       'other', 'reasonable enough', 'neutral', 'reasonable', 'meh',
       'tired but', 'overwhelmed', 'awake', 'alert', 'relaxed',
       'stubborn_brain', 'blink', 'too alert', 'drowsy or sleepy',
       'wired or alert', 'tired but alert', 'melancholic', 'firewall',
       'fizzy'], dtype=object)

In [6]:
all['phase'].unique()

array([nan, 'day', 'adhd_meds', 'custom', 'settling', 'mood'],
      dtype=object)

# Settling

In [7]:
settling = all[(all['phase'] == 'settling') | (all['phase'].isna())]

In [8]:
import pandas as pd

def group(df):
    # Fill missing values in 'version' column with 'n/a'
    df['version'] = df['version'].fillna('n/a')

    # Group by 'version' and count unique 'dayAndNightOf' occurrences
    unique_day_and_night_count = df.groupby('version')['dayAndNightOf'].nunique().reset_index(name='Days')

    # Group by 'version' and count the number of rows for each version
    row_count_per_version = df.groupby('version').size().reset_index(name='Debounced events')

    # Group by 'version' and get the first and last 'dayAndNightOf' occurrences
    first_last_day_and_night = df.groupby('version')['dayAndNightOf'].agg(['first', 'last']).reset_index()

    # Merge the DataFrames on 'version'
    combined_result = pd.merge(unique_day_and_night_count, row_count_per_version, on='version')
    combined_result = pd.merge(combined_result, first_last_day_and_night, on='version')

    return combined_result

group(settling)

C:\Users\graha\AppData\Local\Temp\ipykernel_14944\484858487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['version'] = df['version'].fillna('n/a')


,version,Days,Debounced events,first,last
0,n/a,42,850,2024-10-09,2024-12-12
1,v1,2,8,2024-11-18,2024-11-20
2,v2,13,88,2024-11-21,2024-12-03
3,v3,4,157,2024-12-04,2024-12-07
4,v4,5,189,2024-12-08,2024-12-12


In [9]:
settling['BaseEvent'].unique()

array(['tired', 'wired', 'mid', 'sleepy', 'other', 'awake', 'alert',
       'relaxed', 'too alert', 'drowsy or sleepy', 'wired or alert',
       'tired but alert', 'tired but wired', 'firewall', 'fizzy'],
      dtype=object)

In [10]:
settling['BaseEvent'].value_counts()

BaseEvent
tired               500
other               239
sleepy              207
wired                74
tired but wired      63
mid                  59
wired or alert       55
relaxed              36
drowsy or sleepy     31
alert                12
tired but alert       7
awake                 4
too alert             3
firewall              1
fizzy                 1
Name: count, dtype: int64

In [17]:
interested_version = 'v4'
interesting_settling = settling[settling['version'] == interested_version]
interesting_settling.pivot_table(index='dayAndNightOf', columns='BaseEvent', aggfunc='size', fill_value=0)


BaseEvent,drowsy or sleepy,tired,tired but alert,tired but wired,wired or alert
dayAndNightOf,,,,,
2024-12-08,20,5,0,0,1
2024-12-09,6,18,2,11,0
2024-12-10,2,8,3,32,17
2024-12-11,2,2,2,13,2
2024-12-12,1,0,0,7,35


# Setting event mapping
v1 & v2: Bit confused.  Think v2 introduced 'other'.  But as didn't update Home Assistant (maybe for a while?) have some periods where have that and v2.
v3: Just ignore.  Not enough days and I was repeatedly confused while filling it in.

In [11]:
df = settling.copy()

# Not sure this is correct..
df['version'] = df['version'].replace('n/a', 'v1')

In [12]:
event_counts_per_version = df.pivot_table(index='BaseEvent', columns='version', aggfunc='size', fill_value=0)
event_counts_per_version

version,v1,v2,v3,v4
BaseEvent,,,,
alert,0,0,12,0
awake,0,0,4,0
drowsy or sleepy,0,0,0,31
firewall,1,0,0,0
fizzy,1,0,0,0
mid,33,0,26,0
other,196,43,0,0
relaxed,0,0,36,0
sleepy,172,3,32,0


In [13]:
interested_version = 'v4'
event_counts_per_version[interested_version][event_counts_per_version[interested_version] != 0]


BaseEvent
drowsy or sleepy    31
tired               33
tired but alert      7
tired but wired     63
wired or alert      55
Name: v4, dtype: int64

In [14]:
from models.eeg_states.eeg_states import map_settling_events

mapped = map_settling_events(df)

In [15]:
mapped['MappedEvent'].value_counts()

KeyError: 'MappedEvent'

In [121]:
mapped.pivot_table(index='MappedEvent', columns='version', aggfunc='size', fill_value=0)

version,v1,v2,v4
MappedEvent,,,
Alert,302,43,82
Tired,554,45,60


# Day energy

In [23]:
day = all[all['phase'] == 'day']

In [36]:
group(day)

C:\Users\graha\AppData\Local\Temp\ipykernel_21920\484858487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['version'] = df['version'].fillna('n/a')


,version,Days,Debounced events,first,last
0,v1,8,164,2024-11-15,2024-11-22
1,v2,7,174,2024-11-22,2024-11-28
2,v3,12,255,2024-11-29,2024-12-10
3,v4,1,13,2024-12-10,2024-12-10


# Unknown phase

In [31]:
unknown_phase = all[all['phase'].isna()]
unknown_phase

,event,phase,version,note,source,duration,TimestampUK,SinceUK,dayAndNightOf
0,tired_long,NaN,NaN,NaN,NaN,0 days 00:05:00,2024-10-09 21:22:27.992000+01:00,2024-10-09 21:17:27.992000+01:00,2024-10-09
1,tired,NaN,NaN,NaN,NaN,NaT,2024-10-09 21:28:57.046000+01:00,NaT,2024-10-09
2,tired,NaN,NaN,NaN,NaN,NaT,2024-10-09 21:32:36.824000+01:00,NaT,2024-10-09
3,tired_long,NaN,NaN,NaN,NaN,0 days 00:00:15.951000,2024-10-09 21:32:52.775000+01:00,2024-10-09 21:32:36.824000+01:00,2024-10-09
4,tired,NaN,NaN,NaN,NaN,NaT,2024-10-09 21:33:13.921000+01:00,NaT,2024-10-09
...,...,...,...,...,...,...,...,...,...
1407,other_long,NaN,NaN,NaN,NaN,0 days 00:01:50.822000,2024-12-03 22:47:02.362000+00:00,2024-12-03 22:45:11.540000+00:00,2024-12-03
1408,other_long,NaN,NaN,NaN,NaN,0 days 00:00:37.195000,2024-12-03 22:47:39.557000+00:00,2024-12-03 22:47:02.362000+00:00,2024-12-03
1409,other_long,NaN,NaN,NaN,NaN,0 days 00:00:54.864000,2024-12-03 22:48:34.421000+00:00,2024-12-03 22:47:39.557000+00:00,2024-12-03
1410,other_long,NaN,NaN,NaN,NaN,0 days 00:03:48.977000,2024-12-03 22:52:23.398000+00:00,2024-12-03 22:48:34.421000+00:00,2024-12-03


In [35]:
group(unknown_phase)

C:\Users\graha\AppData\Local\Temp\ipykernel_21920\484858487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['version'] = df['version'].fillna('n/a')


,version,Days,Debounced events,first,last
0,n/a,40,845,2024-10-09,2024-12-06


In [27]:
all['phase'].unique()

array([nan, 'day', 'custom', 'settling'], dtype=object)

# Blinks

In [42]:
blinks = all[all['event'] == 'blink']
blinks

,event,phase,version,note,source,duration,TimestampUK,SinceUK,dayAndNightOf
1580,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:47.158000+00:00,NaT,2024-12-06
1581,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:48.625000+00:00,NaT,2024-12-06
1582,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:49.780000+00:00,NaT,2024-12-06
1583,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:51.035000+00:00,NaT,2024-12-06
1584,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:52.224000+00:00,NaT,2024-12-06
1585,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:53.306000+00:00,NaT,2024-12-06
1586,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:54.369000+00:00,NaT,2024-12-06
1587,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:55.424000+00:00,NaT,2024-12-06
1588,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:56.516000+00:00,NaT,2024-12-06
1589,blink,custom,v1,,NaN,NaT,2024-12-06 21:32:57.573000+00:00,NaT,2024-12-06


In [43]:
group(blinks)

C:\Users\graha\AppData\Local\Temp\ipykernel_21920\484858487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['version'] = df['version'].fillna('n/a')


,version,Days,Debounced events,first,last
0,v1,3,48,2024-12-06,2024-12-10
